In [2]:
import pandas as pd
import os
pd.options.mode.chained_assignment = None

In [3]:
# function to clean data of data frame
def clean_df(df):
    try:
        df.columns = ['Party', 'Member', 'Accomodation - Wellington',
        'Accomodation - Non-Wellington', 'Air Travel', 'Surface Travel',
        'VIP Transport', 'Grand Total', 'Inter-Parliamentary Office of the Clerk', 'Quarter Ending']
    except:
        df.columns = ['Party', 'Member', 'Accomodation - Wellington',
        'Accomodation - Non-Wellington', 'Air Travel', 'Surface Travel',
        'Grand Total', 'Inter-Parliamentary Office of the Clerk', 'Quarter Ending']
        df.insert(6, "VIP Transport", "")


    df['Party'] = df['Party'].fillna(method='ffill')
    df = df[df["Party"].str.contains("Total|[0-9]") == False]
    df = df[df['Member'].notna()]


    df['Member'] = df['Member'].str.replace(' \([0-9]\)', '', regex=True)
    return df

In [4]:
# function to add column with the quarter
def append_quarter(filename, df):
    if filename.split()[-1][0].isnumeric():
        file_quarter = str(filename.split()[-2]+" "+filename.split()[-1]).replace('.xlsx', '')
    else:
        file_quarter = str(filename.split()[-1]+" "+filename.split()[0]).replace('.xlsx', '')
    df['Quarter Ending'] = file_quarter
    return df

In [11]:
# Loop through files in Downloads folder, import clean and append to one file

final_df = pd.DataFrame([], columns=['Party', 'Member', 'Accomodation - Wellington',
    'Accomodation - Non-Wellington', 'Air Travel', 'Surface Travel',
    'VIP Transport', 'Grand Total', 'Inter-Parliamentary Office of the Clerk', 'Quarter Ending'])

for filename2 in os.listdir("Downloads/"):
    if filename2.endswith(".xlsx") and filename2.startswith("2"): 
         df = pd.read_excel ('Downloads/'+filename2, skiprows=2, header=[0,1])
         df = append_quarter(filename2, df)
         df = clean_df(df)
         final_df = pd.concat([df, final_df])
    else:
        pass



In [12]:
def insert_comma(string):
    string = str(string)
    return string[:string.rfind(" ")] + "," + string[string.rfind(" "):]


In [13]:
final_df['Member'] = final_df['Member'].str.replace('*', '', regex=True)
final_df['Member'] = final_df['Member'].str.replace('\\n', '', regex=True)
final_df['Member'] = final_df['Member'].str.strip()
final_df['Member'] = final_df['Member'].str.replace('Te Ururoa', 'Te_Ururoa', regex=True)
final_df['Member'] = final_df['Member'].str.replace('Kanwaljit ,Singh', 'Kanwaljit Singh', regex=True)
final_df['Member'] = final_df['Member'].str.replace('Kanwaljit Singh', 'Kanwaljit_Singh', regex=True)
final_df['Member'] = final_df['Member'].str.replace('Julie Anne', 'Julie-Anne', regex=True)
final_df['Member'] = final_df['Member'].str.replace('Gharahman', 'Ghahraman', regex=True)
final_df['Member'] = final_df['Member'].str.replace(' ,', ',', regex=True)
final_df['Member'] = final_df['Member'].str.replace('Anae Neru', 'Anae_Neru', regex=True)
final_df['Member'] = final_df['Member'].str.replace('Aupito William', 'Aupito_William', regex=True)
final_df['Member'] = final_df['Member'].str.replace('Su\'a, William', 'Aupito_William', regex=True)
final_df['Member'] = final_df['Member'].str.replace('Turei', 'Tūrei', regex=True)
final_df['Member'] = final_df['Member'].str.replace('van, de Molen Tim ', 'van de Molen, Tim', regex=True)
final_df['Member'] = final_df['Member'].str.replace(',', ', ', regex=True)
final_df['Member'] = final_df['Member'].str.replace('  ', ' ', regex=True)
final_df['Member'] = final_df['Member'].apply(insert_comma)
final_df['Member'] = final_df['Member'].str.replace(',,', ',', regex=True)
final_df['Member'] = final_df['Member'].str.replace('Warren-Clarke, Angie', 'Warren-Clark, Angie', regex=True)
final_df['Quarter Ending'] = final_df['Quarter Ending'].str.replace('Dec ', 'December ', regex=True)
final_df['Party'] = final_df['Party'].str.replace('Te Paati Maori', 'Maori', regex=True)
final_df['Party'] = final_df['Party'].str.replace('Maori', 'Te Paati Maori', regex=True)



In [14]:
str_cols = ['Accomodation - Wellington', 'Accomodation - Non-Wellington', 'Air Travel',
'Surface Travel', 'VIP Transport', 'Grand Total',
'Inter-Parliamentary Office of the Clerk']

final_df[str_cols] = final_df[str_cols].replace('-', 0, regex=True)
final_df[str_cols] = final_df[str_cols].replace(' - ', 0, regex=True)
final_df[str_cols] = final_df[str_cols].replace('', 0, regex=True)



In [15]:
# final_df[['Quarter Ending (Month)', 'Quarter Ending (Year)']] = final_df['Quarter Ending'].str.split(' ', 1, expand=True)

final_df['Quarter Ending'] = pd.to_datetime(final_df['Quarter Ending'], format='%B %Y').dt.strftime('%Y-%m-%d')



In [16]:
final_df.to_csv('nz_mp_expenses.csv', index = False)